# Load modules

In [2]:
import pandas as pd

# Import data

In [4]:
booking_details = pd.read_excel('data/booking details-Boston.xlsx')
listings = pd.read_excel('data/listings-Boston.xlsx')
reviews = pd.read_excel('data/reviews-Boston.xlsx')

                listing_id       date  review_stars  revenue
0                 11202183 2016-09-06           1.0     3210
1                 13594886 2016-12-11           4.0     1501
2                 36108417 2021-07-12           2.0     4913
3                 50247613 2022-10-20           5.0      620
4                  3969526 2019-06-16           2.0     4295
...                    ...        ...           ...      ...
174507            25770647 2022-12-07           5.0     4013
174508  727368563191104000 2023-01-19           2.0     3058
174509            29217663 2021-10-17           3.0     3710
174510            42065556 2021-02-21           5.0     2071
174511            12269155 2018-12-07           1.0     3950

[174512 rows x 4 columns]
